#### 영화데이터수집
- 수집처 : 메가박스
- 수집처 URL : https://www.megabox.co.kr/movie
- 수집위치 (웹 네비게이션) : 영화 > 전체영화 > 박스오피스
- 수집데이터 (10개 영화 정보만 수집)
    - 영화제목, 실관람평점, 예매순위, 예매율, 누적관객수, 개별관람평점, 개별관람명내용, 긍정(6점이상), 부정(6점미만)

#### 라이브러리 정의하기

In [1]:
### 웹브라우저 오픈을 위한 라이브러리
# - 브라우저 제어
# - 설치해야 합니다. : pip install selenium
from selenium import webdriver

### 수집된 html 정보에 html 타입으로 의미 부여 및 태그정보 조회(정적)
# from bs4 import BeautifulSoup

### 동적 웹페이지 처리 라이브러리
# - 정적 웹페이지 처리처럼 html.parser 처리를 하지 않아도 라이브러리가 알아서 내부적으로 처리해줌
from selenium.webdriver.common.by import By

### 데이터 처리
import pandas as pd

### 시간 라이브러리
# - 브라우저의 로딩시간이 길어지는 경우에는
#   --> 프로그램의 처리 시간도 늦춰주어야 함
#   --> time.sleep(초)
import time

#### 크롬브라우저 제어하기

In [2]:
driver = webdriver.Chrome()

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="1585f4cd306dadb63e33d337b119adff")>

#### 수집할 웹페이지 URL 정의 및 요청 및 html 코드 수집

In [3]:
### 수집할 웹 페이지 URL
url = "https://www.megabox.co.kr/movie"

### 브라우저를 통해서 요청 및 html 코드 받아오기
driver.get(url)

- selenium 동적 페이지 처리시
    - 별도의 파싱(html.parse) 작업 하지 않아도 됨
    - 바로 태그 경로를 이용해서 데이터 수집

- 태그 정보 조회 함수
    - driver.find_elements(By.CSS_SELECTOR, "태그 경로 패턴")
    - 여러건 조회 시 반환 타입은 list 타입
    - 한 건 조회 시 반환 타입은 html 태그 자체

- 클릭 가능한 태그 : a, input, button 태그 이름은 클릭 가능한 경우
    - a태그의 경우 herf 속성 값에 #만 있는 경우 링크 주소가 없는 경우임
    - 상위 태그에서 링크(페이지 전환)처리한다고 보면 됨

#### 영화 목록 전체 추출하기

- 1. 영화 목록 전체 정보를 추출
    - 클릭 할 수 있는 태그를 찾아서 수집해야 합니다.
- 2. 추출한 각 영화 정보를 클릭하여 페이지 전환
- 3. 전환된 페이지에서 해당 영화 정보 추출
    - : 영화제목, 실관람평점, 예매순위, 예매율, 누적관객수
    - : 영화제목, 개별관람평점, 개별관란명내용, 긍정(관람평점6이상)/부정(관렴평점6미만)
- 4. 전환된 페이지에서 추출이 끝나면 --> 다시 목록으로 페이지 전환
- 5. 1~4를 반복하면서 ... 다음 영화 정보 클릭하여 페이지 전환하면서 반복 처리

In [4]:
# 1. 영화 목록 전체 정보를 추출
#    --> 클릭 할 수 있는 태그를 찾아서 수집해야 함
# #movieList > li > div.movie-list-info
# div.movie-list-info

### 영화 목록 전체 가지고 오기 위한 태그 경로 정의
movie_list_path = "div.movie-list-info"

### 태그 경로에 대한 태그정보 모두 조회하기
# - find_elements 또는 find_element의 조회 결과 내에는 태그가 안보임
#   (각 태그의 메모리 주소 정보만 가지고 있음)
movie_elements = driver.find_elements(By.CSS_SELECTOR, movie_list_path)
print(len(movie_elements))

20


#### 20개 태그 정보(list 타입) 중에, 0번쨰 인덱스 태그 정보로 샘플링

In [5]:
### 클릭 테스트 하기
# - 0번째 태그 정보를 이용해서 실제 클릭 후 페이지 전환이 잘 되는지 확인
# - 클릭(click) 처리 함수 : click() 함수 사용 (클릭 이벤트 함수라고 칭함)

### 반환된 값운 0번째 태그 정보 자체 (리스트 아님)
movie_elements[0]

### 클릭 이벤트 처리 : 클릭 시키기
movie_elements[0].click()

In [6]:
### 다시 목록 페이지로 나오기(이전 페이지로 나오기)
# - 웹 브라우저의 history 정보를 이용
# - history 정보 : 페이지를 클릭하면서 이동한 이력정보를 가지고 있는 변수(브라우저가 관리)
# - execute_script() : javascript 프로그램 처리 함수 (javascript 프로그램 코드 작성 가능)
# - html의 이벤트 처리는 javascript 프로그램으로 처리

### window.history.go(-1)
# - window : html 페이지가 보이는 브라우저 자체
# - history : 페이지 전환에 대한 이력 정보
# - go() : 이동(이전 또는 다음), - 기호는 이전, -1은 이전페이지, -2는 이전-이전페이지
driver.execute_script("window.history.go(-1)")

In [7]:
### 예외처리
# - 외부 연결 중 외부 물리적 연결에 문제가 발생한 경우 처리
# - 보통 윈도우 내부의 파일 읽어들이기, 웹크롤링의 경우 접속하고자 하는 사이트가 중지된 경우 등등
# - 문제가 발생할 부분의 코드를 try 영역에 작성
#   --> 문제가 발생한 경우에 대한 처리 코드는 except 영역에 작성
try :
    # 1. 영화 목록 전체 정보를 추출
    #    --> 클릭 할 수 있는 태그를 찾아서 수집해야 합니다.
    # #movieList > li > div.movie-list-info
    # div.movie-list-info
    
    ### 영화 목록 전체 가지고 오기 위한 태그 경로 정의
    movie_list_path = "div.movie-list-info"
    
    ### 태그 경로에 대한 태그정보 모두 조회하기
    # - find_elements 또는 find_element의 조회 결과 내에는 태그가 안보임
    #   (각 태그의 메모리 주소 정보만 가지고 있음)
    movie_elements = driver.find_elements(By.CSS_SELECTOR, movie_list_path)
    print(len(movie_elements))
    
except :
    print("예외 발생!! : find_elements()의 결과값을 가져오지 못했습니다.")
    movie_elements = []

0


In [8]:
if len(movie_elements) >= 1 :
    ### 클릭 이벤트 처리하기
    movie_elements[0].click()
else :
    print("수집된 영화 목록이 없습니다.")

수집된 영화 목록이 없습니다.


In [9]:
### 웹브라우저 종료하기
driver.quit()

In [10]:
# div.movie-list-info

#### 영화목록 하단의 [더보기] 버튼 처리
- 더보기
    - 더보기 버튼 클릭 시 목록 20개 추가
    - 더보기 버튼이 안보이는 시점에 모든 영화 목록 전체를 가지고 옴
        - 한번에 모든 영화 목록 태그 정보를 수집할 수 있음 (방법 적인 부분으로, 분석가 마음)

- 더보기 버튼 처리 순서
- 1. [더보기] 버튼의 태그 정보를 추출하기 위한 태그 경로(패턴) 찾기
- 2. 찾은 태그를 이용해서 -> [더보기] 클릭 처리하기 -> [더보기] 버튼이 보이지 않을때까지 계속 클릭
- 3. [더보기] 버튼이 안보이는 시점에서 -> 모든 목록이 보임
    - -> 모든 목록이 보이는 시점의 현재 페이지 정보를 인식(현재 페이지 html 코드 수집) 해야
        (클릭 또는 이전/이후에 대한 처리 후에는 전환 후 페이지 html 코드를 다시 인식해야함)

In [11]:
### 크롬브라우저 open 하는 코드를 함수로
# - url을 받아서 요청 및 응답받는 처리까지 모두 수행하는 함수
# - 드라이버 변수를 반환
# 함수이름 : getDriver()

def getDriver(url) :
    new_driver = webdriver.Chrome()
    new_driver.get(url)
    return new_driver

### 크롬부라우저 열기
driver = getDriver("https://www.megabox.co.kr/movie")
driver

In [13]:
# 1. [더보기] 버튼의 태그 정보를 추출하기 위한 태그 경로(패턴) 찾기
# [더보기] 태그 경로 : #btnAddMovie
more_view_path = "#btnAddMovie"

### [더보기] 태그 정보 추출
# 변수명 : more_view_element
more_view_element = driver.find_element(By.CSS_SELECTOR, more_view_path)
more_view_element

<selenium.webdriver.remote.webelement.WebElement (session="ffbd5a3b9ad9090543db570b8a576331", element="f.832E7461E37672498125CD19B0575A87.d.D070BCC972EA657F8A5056FAC455AA63.e.67")>

In [14]:
### 클릭 이벤트 한번 처리해 보기
# - [더보기] 버튼이 안보이는 상태에서, 아래 클릭 코드를 실행하면 오류 발생
try :
    more_view_element.click()
    
except :
    print("[더보기] 버튼이 더 이상 보이지 않습니다.")

### 더보기 버튼 모두 펼쳐 놓는 코드 작성

In [15]:
### 크롬부라우저 열기
driver = getDriver("https://www.megabox.co.kr/movie")
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="be3f31e7da37c38430deb4ccb314c054")>

In [16]:
# 1. [더보기] 버튼의 태그 정보를 추출하기 위한 태그 경로(패턴) 찾기
# [더보기] 태그 경로 : #btnAddMovie
more_view_path = "#btnAddMovie"

### [더보기] 태그 정보 추출해 주세요..
# 변수명 : more_view_element
more_view_element = driver.find_element(By.CSS_SELECTOR, more_view_path)

In [17]:
### [더보기] 버튼이 안보일 때 까지 클릭해서 모든 목록을 펼처 놓기 위한 코드 작성

cnt = 0
while True :
    try :
        ### 더보기 클릭 처리
        more_view_element.click()
        
        ### 브라우저에서 클릭 후 로딩하는 시간 동안 프로그램은 잠시 대기 처리
        # - 1초 대기
        time.sleep(1)

        ### cnt 1씩 증가시키기
        # or cnt += 1
        cnt = cnt + 1
        
    except :
        print("[더보기] 버튼이 더 이상 보이지 않습니다.")
        break

### 총 반복횟수 출력하기
print(f"총 반복 횟수 : {cnt}")

[더보기] 버튼이 더 이상 보이지 않습니다.
총 반복 횟수 : 5


##### 모든 목록이 보이는 시점의 현재 페이지 정보를 인식(현재 페이지 html 코드 수집) 해야 함
##### (클릭 또는 이전/이후에 대한 처리 후에는 전환 후 페이지 html 코드를 다시 인식해야함)

In [18]:
### window_handles : 페이지 변경 사항에 대한 정보를 담고 있음
# - 클릭 후 페이지 변경이 되면, 해당 window_handles 리스트에 정보가 하나씩 추가됨
# - 마지막 리스트 값을 사용
# - window_handles의 결과값은 리스트 타입
page_handle = driver.window_handles[-1]
page_handle

### window_handles의 마지막 값을 브라우저, 즉 window에 다시 반영 시켜야 함
driver.switch_to.window(page_handle)

In [19]:
### 위 코드를 함수로 정의해서 사용하기
# - 함수 이름 : setWindowPage
def setWindowPage(p_driver) :
    ### window_handles : 페이지 변경 사항에 대한 정보를 담고 있음
    # - 클릭 후 페이지 변경이 되면, 해당 window_handles 리스트에 정보가 하나씩 추가
    # - 마지막 리스트 값을 사용하면 됨
    # - window_handles의 결과값은 리스트 타입
    page_handle = p_driver.window_handles[-1]
    page_handle
    
    ### window_handles의 마지막 값을 브라우저, 즉 window에 다시 반영 시켜야 함
    p_driver.switch_to.window(page_handle)

    time.sleep(1)

In [20]:
### 현재 변경된 페이지 인식하기
setWindowPage(driver)

#### [더보기] 이후 전체 목록 정보 조회하기

In [21]:
try :
    # 1. 영화 목록 전체 정보를 추출
    #    --> 클릭 할 수 있는 태그를 찾아서 수집해야 함
    ### 영화 목록 전체 가지고 오기 위한 태그 경로 정의
    movie_list_path = "div.movie-list-info"
    
    ### 태그 경로에 대한 태그정보 모두 조회하기
    # - find_elements 또는 find_element의 조회 결과 내에는 태그가 안보임
    #   (각 태그의 메모리 주소 정보만 가지고 있음)
    movie_elements = driver.find_elements(By.CSS_SELECTOR, movie_list_path)
    print(len(movie_elements))
    
except :
    print("예외 발생!! : find_elements()의 결과값을 가져오지 못했습니다.")
    movie_elements = []

103


#### 영화목록 1개를 기준으로 샘플링 하기

In [22]:
### 영화목록 1개 추출하여, 클릭해서 정보 페이지로 전환하기
movie_elements[4].click()

In [23]:
### 페이지 전환 이후 전환된 페이지를 window(브라우저)에 인식 시키기
setWindowPage(driver)

In [24]:
### 영화 상세 정보 추출하기
#  : 영화제목, 실관람평점, 예매순위, 예매율, 누적관객수
#  : 영화제목, 개별관람평점, 개별관란명내용, 긍정(관람평점6이상)/부정(관렴평점6미만)

In [25]:
# - [영화 제목] 추출하기
# - 영화제목이 있는 태그 경로 정의하기
title_path = "#contents > div.movie-detail-page > div.movie-detail-cont > p.title"

### 영화제목이 있는 태그 정보 추출하기
title_element = driver.find_element(By.CSS_SELECTOR, title_path)

### 추출한 태그에서 영화제목 text 추출하여 출력하기
title = title_element.text
print(f"영화제목 : {title}")

영화제목 : 캡틴 아메리카: 브레이브 뉴 월드


In [26]:
### [실관람평] 추출하기
# - 실관람평 태그 경로 정의
score_path = "#mainMegaScore > p > em"

# - 실관람평 태그 추출하기
score_element = driver.find_element(By.CSS_SELECTOR, score_path)

# - 실관람평 텍스트 추출하기
score = score_element.text
print(f"실관람평 : {score}")

실관람평 : 8.3


In [27]:
### [예매순위] 및 [예매율] 추출하기
# - 예매순위 텍스트값을 담을 변수명 : level
# - 예매율 텍스트값을 담을 변수명 : rate
#    -> 예매율은 selector가 안될겁니다. (selector 되는 시점의 태그를 찾아야 합니다.)
# - hint : 문자열.split("구분자") 함수 사용 -> 함수 결과는 구분된 값을 리스트(list)로 반환

rate_level_path = "#contents > div.movie-detail-page > div.movie-detail-cont > div.info > div.rate > p.cont"
rate_level_element = driver.find_element(By.CSS_SELECTOR, rate_level_path)
rate_level = rate_level_element.text
# '5위 (4.7%)'
### 문자열에서 띄어쓰기로 구분하여 리스트로 만들기 : split() 함수 사용
# - 띄어쓰기 구분자는 생략가능(기본값으로 사용됨)
temp_split = rate_level.split(" ")
temp_split

### 예매순위 추출하기
level = temp_split[0][:-1]
print(f"예매순위 : {level}")

### 예매율 추출하기
rate = temp_split[1][1:][:-2]
print(f"예매율 : {rate}")

예매순위 : 5
예매율 : 4.2


In [28]:
### 누적관객수 추출하기
# - 값을 저장할 변수명 : audiance
# - 최종 값에서 콤마(,)는 제외하고 숫자만 추출하여 변수에 저장하기
audiance_path = "#contents > div.movie-detail-page > div.movie-detail-cont > "
audiance_path += "div.info > div.audience > p > em"

audiance_element = driver.find_element(By.CSS_SELECTOR, audiance_path)

# 결과값에서 '1,312,372' -> 콤마 제거 -> replace() 함수 사용
audiance = audiance_element.text.replace(",", "")
audiance

'1343688'

#### [실관람평] 탭을 클릭하여 개별 관련 평점 및 평내용 추출하기

In [29]:
### [실관람평] 탭 클릭을 위한 태그 정보 추출하기
# - 탭 경로 정의
movie_tab_path = "#contentData > div:nth-child(5) > div.tab-list.fixed > ul "
movie_tab_path += "> li:nth-child(2) > a"

# - 탭 태그 정보 추출
movie_tab_element = driver.find_element(By.CSS_SELECTOR, movie_tab_path)

# - 탭 태그 텍스트 추출해 보기
print(movie_tab_element.text)

# - 탭 태그의 링크 주소(url) 추출해 보기
#  -- a = {href:"url"}
#  --> BeautifulSoup 방식 : 태그변수["href"] 
#  --> selenium 방식 : 속성의 값 추출하는 함수 : 태그변수.get_attribute("href")
print(movie_tab_element.get_attribute("href"))

실관람평
https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieOneDetail.do


In [30]:
### [실관람평] 탭 클릭 처리하기
movie_tab_element.click()

In [31]:
### 클릭 이벤트 처리 후에는 브라우저에 현재 페이지 정보 인식시키기
setWindowPage(driver)

In [32]:
### [개별 관람평점] 추출하기
# - 개별관람평점 태그 경로 정의
# div.story-area > div.story-box > div > div.story-cont > div.story-point > span
# div.story-area > div.story-box > div > div.story-cont > div.story-point > span
use_score_path = "div.story-area > div.story-box > div > div.story-cont > div.story-point > span"

# - 개별관람평점이 있는 모든 태그 정보 추출하기
use_score_elements = driver.find_elements(By.CSS_SELECTOR, use_score_path)
print("개별관람평점 전체 태그 조회 갯수 : ", len(use_score_elements))

# - 0번째 태그의 텍스트 값 추출하여 출력해 보기
use_score = use_score_elements[0].text
print(f"0번째 개별관람평 평점 : {use_score}")

개별관람평점 전체 태그 조회 갯수 :  10
0번째 개별관람평 평점 : 8


In [33]:
### 개별 관람평 내용 모두 추출하기
# - 개별관람평 내용 태그 경로
# div.story-area > div.story-box > div > div.story-cont > div.story-txt
# div.story-area > div.story-box > div > div.story-cont > div.story-txt
use_content_path = "div.story-area > div.story-box > div > div.story-cont > div.story-txt"

# - 개별관람평 내용 태그 정보 추출
use_content_elements = driver.find_elements(By.CSS_SELECTOR, use_content_path)
print("개별관렴평 내용 태그 갯수 : ", len(use_content_elements))

# - 0번째 태그의 개별관람평 내용 텍스트 출력해 보기
use_content = use_content_elements[0].text
print(f"0번째 개별관람평 내용 : {use_content}")

개별관렴평 내용 태그 갯수 :  10
0번째 개별관람평 내용 : 확실히 어거지 스토리인듯


In [34]:
### 현재 영화정보에 대한 수집이 완료되면, 
# - 전체 목록 페이지로 돌아가지 : 이전페이지로 가기
driver.execute_script("window.history.go(-1)")
# - 이전페이지 로딩되는 시간동안 프로그램 실행 잠시 대기
time.sleep(1)

In [35]:
### 브라우저 종료
driver.quit()